In [5]:
import cv2
import numpy as np

from tool.utils import *
from tool.torch_utils import do_detect
from tool.darknet2pytorch import Darknet


def call_yolov4(cfgfile='../yolov4.cfg', weightfile='../yolov4.weights', use_cuda=True):
    m = Darknet(cfgfile)
    m.load_weights(weightfile)

    if use_cuda:
        m.cuda().eval()
    else:
        m.eval()

    return m


class yolov4_trck():
    def __init__(self, m):
        self.m = m

    def object_detection(self, frame, ret):
        if ret == False:
            print(ret)
            return 'no_frame'
        else:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)        
            sized = cv2.resize(frame, (512, 512))
            sized = cv2.cvtColor(sized, cv2.COLOR_BGR2RGB)

            #### Start detection using do_detect() function
    #             start = time.time()
            ######### output must be boxes[0], which contains tbrl, confidence level, and class number
            boxes = do_detect(self.m, sized, 0.4, 0.6, True)
    #             print(type(boxes), len(boxes[0]), ': number of detected cars', boxes[0])
    #             finish = time.time()
    #             print('yolo elapsed in: %f sec' % (finish - start))
            return boxes[0]



    def run_yolov4(self, frame, ret):
        return self.object_detection(frame, ret)


In [6]:
import cv2
import numpy as np

from deep_sort.deepsort import *

import torch
# import torchvision

import time



def call_deepsort(wt_path='../model640.pt', use_cuda=True):
    m_deepsort = torch.load(wt_path)

    if use_cuda:
        m_deepsort.cuda().eval()
    else:
        m_deepsort.eval()

    return m_deepsort


class run_class():
    def __init__(self, DSort, road=20*3*3, fps=60, road_length=20):
        self.DSort = DSort

        self.outgoing = []
        self.incoming = []

        self.out_occupied = 0
        self.in_occupied = 0
        self.road = road

        self.speed_outgoing = {}
        self.speed_incoming = {}
        self.fps = fps
        self.d = road_length
        self.out_speed = {}
        self.in_speed = {}


    def flow(self, t, b, r, l, id_num):
        if t < 540 and b > 540:     ### if a box acrosses the bottom (yellow) line
            if r < 540:             ### and the box on the left side (enter RoI)
                if id_num not in self.outgoing:
                    self.outgoing.append(id_num)
            elif l > 670:           ### and the box on the right side (exit RoI)
                if id_num not in self.incoming:
                    self.incoming.append(id_num)



    def density(self, t, b, r, l, outclass):
        if t < 540 and b > 340:
            if r < 540:
                if outclass == 2:  ### car
                    self.out_occupied += 4.5*1.7
                elif outclass == 5:  ### bus
                    self.out_occupied += 13*2.55
                elif outclass == 7:  ### truck
                    self.out_occupied += 5.5*2

            elif l > 670:
                if outclass == 2:  ### car
                    self.in_occupied += 4.5*1.7
                elif outclass == 5:  ### bus
                    self.in_occupied += 13*2.55
                elif outclass == 7:  ### truck
                    self.in_occupied += 5.5*2


    def speed(self, t, b, r, l, id_num):
        if t < 540 and b > 540:     ### yellow line
            if r < 540:             ### come into yellow line
                if id_num not in self.speed_outgoing:
                    self.speed_outgoing[id_num] = [1]    ### the vehicle is getting further, and firstly captured
                else:
                    self.speed_outgoing[id_num][0] += 1   ### the vehicle is gettig further, and captured multiple times
            elif l > 670:           ### get out from yellow line
                if id_num not in self.speed_incoming:   ### the vehicle is getting closer, and don't know when it first get into the RoI
                    pass
                elif len(self.speed_incoming[id_num]) < 2:   ### the vehicle is getting closer, and now exiting the RoI -- need to calculate speed
                    self.speed_incoming[id_num][0] += 1
                    self.speed_incoming[id_num].append(1)

        if t < 340 and b > 340:     ### blue line
            if r < 540:             ### get out from blue line
                if id_num not in self.speed_outgoing:    ### the vehicle is getting further, and don't know when it firstly get into the RoI
                    pass
                elif len(self.speed_outgoing[id_num]) < 2:   ### the vehicle is getting further, and now exiting the RoI
                    self.speed_outgoing[id_num][0] += 1
                    self.speed_outgoing[id_num].append(1)
            elif l > 670:           ### come into blue line
                if id_num not in self.speed_incoming:
                    self.speed_incoming[id_num] = [1]   ### the vehicle is getting closer, and firstly captured
                else:
                    self.speed_incoming[id_num][0] += 1   ### the vehicle is getting closer, and cpatured multiple times


        if b < 340 and r < 540:       #### outgoing -- the vehicle exited the RoI -- calculate speed
            if id_num in self.speed_outgoing:
#                 print(self.speed_outgoing)
                delta_t = self.speed_outgoing[id_num][0] * (1/self.fps)
                delta_d = self.d
                self.out_speed[id_num] = round((delta_d/delta_t)*3.6, 2)  ### change unit from m/s to km/h by multiplying 3.6
#                 print('>> outgoing speed: ', self.out_speed[id_num], 'km/h')
                self.speed_outgoing.pop(id_num)
        if t > 540 and l > 670:        #### incoming -- the vehicle exited the RoI -- calculate speedl
            if id_num in self.speed_incoming:
#                 print(self.speed_incoming)
                delta_t = self.speed_incoming[id_num][0] * (1/self.fps)
                delta_d = self.d
                self.in_speed[id_num] = round((delta_d/delta_t)*3.6, 2)  ### change unit from m/s to km/h by multiplying 3.6
#                 print('>>> incoming speed: ', self.in_speed[id_num], 'km/h')
                self.speed_incoming.pop(id_num)




    def processing(self, boxes, class_names, frame, ret):
        if ret == False:
            print(ret, 'no_frame')
            return
        else:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = frame.astype(np.uint8)

            tracker, detections_class = self.DSort.a_run_deep_sort(frame, boxes)

            for track in tracker.tracks:
#                 print('track.is_confirmed(): ', track.is_confirmed())
#                 print('track.time_since_update: ', track.time_since_update)
                if not track.is_confirmed() or track.time_since_update > 1:
                    continue

                bbox = track.to_tlbr() #Get the corrected/predicted bounding box
                id_num = str(track.track_id) #Get the ID for the particular track.
                features = track.features #Get the feature vector corresponding to the detection.

                l = bbox[0]  ## x1
                t = bbox[1]  ## y1
                r = bbox[2]  ## x2
                b = bbox[3]  ## y2

                self.flow(t, b, r, l, id_num)
                self.density(t, b, r, l, track.outclass)
                self.speed(t, b, r, l, id_num)



    def run_dsort(self, boxes, class_names, frame, ret):
        self.processing(boxes, class_names, frame, ret)


In [11]:
# from queue import Queue

# ### initiate queue
# q = Queue(maxsize = 1)

### video params
video_path='../tracking_record1.mov'

cap = cv2.VideoCapture(video_path)
cvfps = cap.get(cv2.CAP_PROP_FPS)
cvfps = 30
print('fps:  ', cvfps)

width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # float
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) # float

### ML models
#### yolo model
m = call_yolov4()
ODetect = yolov4_trck(m)

namesfile='detection/coco.names'
num_classes = m.num_classes
class_names = load_class_names(namesfile)
# print(class_names)

#### deepsort model
m_deepsort = call_deepsort()
DSort = deepsort_rbc(m_deepsort, width, height)
RClass = run_class(DSort, road=20*3*3, fps=cvfps, road_length=20)


#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
count = 0
test = False
total_frames = 0
while True:
    count += 1
    print(total_frames)
    if count == 60*8+1:
        break

    ret, frame = cap.read()

    if count % 2 == 1:
        continue
    total_frames += 1
    result = ODetect.run_yolov4(frame, ret)
    RClass.run_dsort(result, class_names, frame, ret)

#         print('a: ', round(RClass.out_occupied/RClass.road*100, 2))
#         print('b: ', round(RClass.in_occupied/RClass.road*100, 2))

#         RClass.out_occupied = 0
#         RClass.in_occupied = 0

### Split results (averaged in sec -- or total length of the video)
##### traffic flow
total_sec = total_frames/cvfps
print(total_frames, cvfps, total_sec)
print(len(RClass.outgoing)/total_sec, len(RClass.incoming)/total_sec)
RClass.outgoing = []
RClass.incoming = []

##### traffic density
print('a: ', round(RClass.out_occupied/(RClass.road*total_frames)*100, 2))
print('b: ', round(RClass.in_occupied/(RClass.road*total_frames)*100, 2))
RClass.out_occupied = 0
RClass.in_occupied = 0

##### traffic speed
s = [0,0,0,0]
for k, v in RClass.out_speed.items():
    s[0] += 1
    s[1] += v
for k, v in RClass.in_speed.items():
    s[2] += 1
    s[3] += v
print(s)
print('speed out: ', round(s[1]/s[0], 2))
print('speed in: ', round(s[3]/s[2], 2))

print('stop plugin')
cap.release()


cap.release()

fps:   30
convalution havn't activate linear
convalution havn't activate linear
convalution havn't activate linear
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
73
73
74
74
75
75
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
83
84
84
85
85
86
86
87
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
103
103
104
104
105
105
106
106
107
107
108
108
109
109
110
110
111
111
112
112
113
113
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
121
122
122
123
123
124
124
125
125
126
126
127
127
128
128
129
129
130
130
131
131
132
132
133
133
134
134
135
135
136
136
137
137
1

In [ ]:
from time import sleep

In [ ]:
def run_yolov4(kill_queue, output_queue):
    video_path='../tracking_record1.mov'

    cap = cv2.VideoCapture(video_path)
    cvfps = cap.get(cv2.CAP_PROP_FPS)
    cvfps = 30
    print('fps:  ', cvfps)

    width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # float
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) # float

    m = call_yolov4()
    ODetect = yolov4_trck(m)

    namesfile='detection/coco.names'
    num_classes = m.num_classes
    class_names = load_class_names(namesfile)

    while kill_queue.empty():
        if output_queue.full():
            sleep(0.1)
            continue

        ret, frame = cap.read()
        if ret == False:
            break
        ret, frame = cap.read()
        if ret == False:
            break

        output_queue.put((ODetect.run_yolov4(frame, ret), frame, ret))

    kill_queue.put(1)
    cap.release()

In [7]:
def run_deepsort(kill_queue, input_queue):
    video_path='../tracking_record1.mov'

    cap = cv2.VideoCapture(video_path)
    cvfps = cap.get(cv2.CAP_PROP_FPS)
    cvfps = 30
    print('fps:  ', cvfps)

    width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # float
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) # float
    cap.release()


    namesfile='detection/coco.names'
    class_names = load_class_names(namesfile)
    # print(class_names)

    #### deepsort model
    m_deepsort = call_deepsort()
    DSort = deepsort_rbc(m_deepsort, width, height)
    RClass = run_class(DSort, road=20*3*3, fps=cvfps, road_length=20)

    total_frames = 0
    while kill_queue.empty():
        if RClass.run_dsort(detection, class_names, frame, ret).empty():
            sleep(0.1)
            continue
        total_frames += 1
        detection, frame, ret = input_queue.get()
        RClass.run_dsort(detection, class_names, frame, ret)



    ### Split results (averaged in sec -- or total length of the video)
    ##### traffic flow
    total_sec = total_frames/30
    print(total_frames, cvfps, total_sec)
    print(len(RClass.outgoing)/total_sec, len(RClass.incoming)/total_sec)
    RClass.outgoing = []
    RClass.incoming = []

    ##### traffic density
    print('a: ', round(RClass.out_occupied/(RClass.road*total_frames)*100, 2))
    print('b: ', round(RClass.in_occupied/(RClass.road*total_frames)*100, 2))
    RClass.out_occupied = 0
    RClass.in_occupied = 0

    ##### traffic speed
    s = [0,0,0,0]
    for k, v in RClass.out_speed.items():
        s[0] += 1
        s[1] += v
    for k, v in RClass.in_speed.items():
        s[2] += 1
        s[3] += v
    print(s)
    print('speed out: ', round(s[1]/s[0], 2))
    print('speed in: ', round(s[3]/s[2], 2))

    print('stop plugin')

In [8]:
from multiprocessing import Process, Queue, Value

message_queue = Queue(maxsize=1)
kill_queue = Queue(maxsize=1)

yolo_process = Process(target=run_yolov4, args=(kill_queue, message_queue,))
dsort_process = Process(target=run_deepsort, args=(kill_queue, message_queue,))

yolo_process.start()
dsort_process.start()

try:
    sleep(1)
except KeyboardInterrupt:
    pass
finally:
    kill_queue.put(0)
    yolo_process.join()
    dsort_process.join()
    

NameError: name 'run_yolov4' is not defined

In [14]:
a = [1,2,3]
b = [4,5,6]
a = a+b
a

[1, 2, 3, 4, 5, 6]

In [17]:
import time
a = time.time()
print(a)

import datetime
datetime.datetime.fromtimestamp(a).strftime('%c')

1616663135.1179547


'Thu Mar 25 09:05:35 2021'